In [3]:
# import modules
import os
from Bio import SeqIO
from Bio import Entrez
#Entrez.email = "brendan.jeffrey@nih.org"  # Always tell NCBI who you are
#filename = "b_burgdorferi_B31_genome.gbk"
#if not os.path.isfile(filename):
#    # Downloading...
#    net_handle = Entrez.efetch(db="nucleotide", id="AE000783.1", rettype="gb", retmode="text")
#    out_handle = open(filename, "w")
#    out_handle.write(net_handle.read())
#    out_handle.close()
#    net_handle.close()
#    print("Genbank Saved")

In [49]:
# opening files
in_genbank = "./b_burgdorferi_B31_genome.gbk"
in_deseq_res = "../data/sb01_ph68C_ph76C_diffs.txt"
out_anno_res = open("../data/sb01_ph68C_ph76C_diffs_anno.txt", 'w')

In [50]:
# write header for outfile
out_anno_res.write("locus_tag\tgene\tproduct\tbaseMean\tlog2FoldChange\tlfcSE\tstat\tpvalue\tpadj\n")

70

In [51]:
cars = [["honda", "toyota"], ["ford" ,"chevy"]]

for country in cars:
    for make in country:
        print(make)

honda
toyota
ford
chevy


In [52]:
# extract gene names and gene products, populate dicts
gene_dict = {}
product_dict = {}

# parse genbank file, populate dicts with gene names and products
with open(in_genbank):
    full_record = SeqIO.parse(in_genbank, "genbank")
    for record in full_record:
        # investigate record object, tab autocompletion
        #print(record)
        #print(record.description)
        
        # loop though each feature  of each record
        for feature in record.features:
            # investigate feature object
            #print(feature)
            
            # only look at coding sequences
            if "CDS" in feature.type:
                # extract locus_tags , products
                locus_tag = feature.qualifiers.get('locus_tag')[0]
                product = feature.qualifiers.get('product')[0]
            
                
                # populate dictionaries
                if feature.qualifiers.get('gene'):
                    gene = feature.qualifiers.get('gene')[0]
                else:
                    gene = "NA"
                product_dict[locus_tag] = product
                gene_dict[locus_tag] = gene

In [53]:
#print(product_dict)

In [54]:
# add gene name and gene product to results
with open(in_deseq_res) as f:
    # skip the header
    next(f)
    for line in f:
        line = line.rstrip("\n")
        locus_tag, baseMean, log2FoldChange, lfcSE, stat, pvalue, padj = line.split("\t")
        #print(locus_tag)
        if locus_tag in product_dict.keys():
            out_anno_res.write("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\t{8}\n".format(locus_tag, gene_dict[locus_tag], product_dict[locus_tag], baseMean, log2FoldChange, lfcSE, stat, pvalue, padj))
        else:
            out_anno_res.write("{0}\tNA\tNA\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\n".format(locus_tag, baseMean, log2FoldChange, lfcSE, stat, pvalue, padj))

out_anno_res.close()